In [6]:
import threading
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from openpyxl import load_workbook
from init_driver import init_driver
from count_dropdown_itens import count_dropdown_itens
from list_helper import list_pop, list_receiver
from get_daily_data import get_daily_data

wb = load_workbook('../BD_CADASTRO_NUMERADO_AGO_TESTE.xlsx')
ws, ws2 = wb['Fundos'], wb['Fundos_Cota']
lista_cnpj = []

for row in ws.iter_rows(values_only=True):
    cnpj = row[1]
    lista_cnpj.append(cnpj)

for row in ws2.iter_rows(values_only=True):
    cnpj = row[1] 
    lista_cnpj.append(cnpj)

print("Coleta de CNPJs finalizada")

dados_diarios_list = []

def get_inside(cnpj, index, item_to_finish):
    try:
        print(f"Thread {index+1} iniciada para o CNPJ {cnpj}")
        driver = init_driver(index)
        print(f"Driver da Thread {index+1} iniciado com sucesso")

        #pesquisa e troca para o frame correto sem precisar declarar uma variável nova no processo
        WebDriverWait(driver, 10).until(
        EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
        )

        search_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//input[@id='txtCNPJNome']"))
        )

        search_input.clear()
        
        search_input.send_keys(cnpj)
        search_input.send_keys(Keys.ENTER)
        print(f"Thread {index+1} pesquisou o CNPJ {cnpj}")

        try:
            find_name_to_store = WebDriverWait(driver, 1).until(
                EC.presence_of_element_located((By.XPATH, "//a[@id='ddlFundos__ctl1_Linkbutton4']"))
            )
        except TimeoutException:
            find_name_to_store = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//a[@id='ddlFundos__ctl0_lnkbtn1']"))
            )

        stored_name = find_name_to_store.text
        find_name_to_store.click()

        driver.switch_to.default_content()

        #pesquisa e troca para o frame correto sem precisar declarar uma variável nova no processo
        WebDriverWait(driver, 10).until(
            EC.frame_to_be_available_and_switch_to_it((By.XPATH, "//frame[@name='Main']"))
        )

        dados_diarios = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//a[@id='Hyperlink2']"))
        )
        dados_diarios.click()
        print(f"Thread {index+1} clicou em dados diários")

        driver.switch_to.default_content()

        #pop-up com mensagem irrelevante aparece e precisa ser fechado.
        try:
            alert = WebDriverWait(driver, 3).until(EC.alert_is_present())
            alert.accept()
        except:
            pass

        dados_diarios_list.extend(get_daily_data(index, driver, cnpj, stored_name, item_to_finish))

        
    except TimeoutException:
        print(f"Erro na Thread {index+1}")
        driver.quit()
        print(f"Thread {index+1} reiniciando...")
        get_inside(cnpj, index, item_to_finish)
    driver.quit()

#list_cnpj_teste = ["58.878.941/0001-02", "36.248.874/0001-00", "51.017.442/0001-81", "57.270.020/0001-08"]
list_cnpj_teste = ["58.878.941/0001-02"]

threads = []

# Criando threads em blocos de 8
for cnpj in list_cnpj_teste:
    print("Coleta tamanho do dropdown iniciada")
    tamanho = count_dropdown_itens(cnpj)
    print("Tamanho do dropdown coletado com sucesso")
    list_to_finish = list(range(tamanho))
    list_receiver(list_to_finish)
    for index in range(6):
        try:
            var = list_pop()
            thread = threading.Thread(target=get_inside, args=(cnpj, index, var))
            print(list_to_finish)
            threads.append(thread)
            thread.start()
        except IndexError:
            pass

# Aguardando todas as threads terminarem
for thread in threads:
    thread.join()

print("Todas as Threads foram finalizadas")

Coleta de CNPJs finalizada
Coleta tamanho do dropdown iniciada
Tamanho do dropdown coletado com sucesso
[1]
Thread 1 iniciada para o CNPJ 58.878.941/0001-02
[]
Thread 2 iniciada para o CNPJ 58.878.941/0001-02
Driver da Thread 2 iniciado com sucesso
Driver da Thread 1 iniciado com sucesso
Thread 1 pesquisou o CNPJ 58.878.941/0001-02
Thread 2 pesquisou o CNPJ 58.878.941/0001-02
Thread 2 clicou em dados diários
Thread 2 iniciando coleta de dados diários
[{'Cnpj': '58.878.941/0001-02', 'Nome': '051 MHX ALTERNATIVOS FUNDO DE INVESTIMENTO FINANCEIRO MULTIMERCADO CRÉDITO PRIVADO RESP LIMITADA', 'Mês': '01/2025', 'Dia': '31', 'Quota': '1,0019343', 'Patrimônio Líquido': '155.520.776,59', 'Número de Cotistas': '1'}]
Thread 2 finalizou coleta de dados diários
Thread 2 finalizada
Erro na Thread 1
Thread 1 reiniciando...
Thread 1 iniciada para o CNPJ 58.878.941/0001-02
Driver da Thread 1 iniciado com sucesso
Thread 1 pesquisou o CNPJ 58.878.941/0001-02
Thread 1 clicou em dados diários
Thread 1 inic

In [8]:
import pandas as pd
testing = pd.DataFrame(dados_diarios_list)
testing['Mês'] = pd.to_datetime(testing['Mês'], format='%m/%Y')
testing_sorted = testing.sort_values(by='Mês', ignore_index=True)

testing_sorted


,Cnpj,Nome,Mês,Dia,Quota,Patrimônio Líquido,Número de Cotistas
0,58.878.941/0001-02,051 MHX ALTERNATIVOS FUNDO DE INVESTIMENTO FIN...,2025-01-01,31,"1,0019343","155.520.776,59",1
1,58.878.941/0001-02,051 MHX ALTERNATIVOS FUNDO DE INVESTIMENTO FIN...,2025-02-01,17,"1,0035021","155.764.130,65",1


In [5]:
testing_sorted = testing.sort_values(by=['Cnpj', 'Mês'], ignore_index=True)
testing_sorted

,Cnpj,Nome,Mês,Dia,Quota,Patrimônio Líquido,Número de Cotistas
0,36.248.874/0001-00,051 CRÉDITO 90 FUNDO DE INVESTIMENTO FINANCEIR...,2024-11-01,29,"1,5316007","368.876.093,12",56
1,36.248.874/0001-00,051 CRÉDITO 90 FUNDO DE INVESTIMENTO FINANCEIR...,2024-12-01,31,"1,5486839","372.740.265,23",54
2,36.248.874/0001-00,051 CRÉDITO 90 FUNDO DE INVESTIMENTO FINANCEIR...,2025-01-01,31,"1,566572","373.149.451,52",55
3,36.248.874/0001-00,051 CRÉDITO 90 FUNDO DE INVESTIMENTO FINANCEIR...,2025-02-01,18,"1,5767656","375.594.588,28",55
4,51.017.442/0001-81,051 CHIMERA FEEDER FUNDO DE INVESTIMENTO FINAN...,2024-10-01,31,"1.195,32948236","13.024.996,62",6
5,51.017.442/0001-81,051 CHIMERA FEEDER FUNDO DE INVESTIMENTO FINAN...,2024-11-01,29,"1.204,61210793","13.126.145,44",6
6,51.017.442/0001-81,051 CHIMERA FEEDER FUNDO DE INVESTIMENTO FINAN...,2024-12-01,31,"1.253,2158189","13.935.343,64",6
7,51.017.442/0001-81,051 CHIMERA FEEDER FUNDO DE INVESTIMENTO FINAN...,2025-01-01,31,"1.277,94733581","14.985.526,18",6
8,51.017.442/0001-81,051 CHIMERA FEEDER FUNDO DE INVESTIMENTO FINAN...,2025-02-01,18,"1.288,74597833","15.112.153,73",6
9,57.270.020/0001-08,1295 ESTRUTURADO FUNDO DE INVESTIMENTO MULTIME...,2024-10-01,31,"1,00364209","111.210.122,22",1


In [ ]:
testing_sorted.to_excel('teste.xlsx', index=False)

In [ ]:
testing_sorted = testing.sort_values(by='Mês', ignore_index=True)
testing_sorted

In [ ]:
tamanho = 6
to_int = int(tamanho)

In [ ]:
a = list(range(to_int))

In [ ]:
b = a.pop(0)
print(b)

In [ ]:
import psutil

def kill_webdrivers():
    for process in psutil.process_iter(attrs=['pid', 'name']):
        if process.info['name'] and ('chromedriver' in process.info['name'].lower() or 'geckodriver' in process.info['name'].lower()):
            print(f"Matando processo {process.info['name']} (PID: {process.info['pid']})")
            psutil.Process(process.info['pid']).terminate()  # Encerra o processo

kill_webdrivers()
